In [65]:
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') 
# Above is a special style template for matplotlib, highly useful for visualizing time series data
%matplotlib inline
from pylab import rcParams
from plotly import tools
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from numpy.random import normal, seed
from scipy.stats import norm
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.arima_model import ARIMA
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [67]:
df=pd.read_csv("Amazon_Daily_Data.csv")
df.head()
df=df.drop(columns='Unnamed: 0')
df

,Name,Laptop ID,Price,Date
0,Dell Inspiron 3593 15.6-inch FHD Laptop (10th ...,B089QMC6M1,39990,13/11
1,Lenovo Legion 5i 10th Gen Intel Core i7 15.6 i...,B08CQ8S6VD,82990,13/11
2,DELL Inspiron 3593 15.6-inch FHD Laptop (10th ...,B08LC55QJ1,35490,13/11
3,HP 14s core i5 10th Gen 14 inch FHD Laptop (8 ...,B08428CSZF,59990,13/11
4,Dell Inspiron 3595 15.6-inch HD Laptop (A9-942...,B0839L8XW1,28490,13/11
...,...,...,...,...
8966,HP 645G4 R7-2700U 14 8GB/256 PC,B07DPGBZ97,150700,18/11
8967,HP Smart Buy Z4 G4 WKSTN I7-7800X,B07C6QCFM5,217875,18/11
8968,"HP EliteBook 840 G5 Notebook, 14"" 4K IPS Ultra...",B07B8QK7PP,203641,18/11
8969,"HP Stream 14 Inch Laptop (2018, Intel Celeron ...",B078T74GKF,73410,18/11


In [68]:
z=df['Date']
for i in range(len(z)):
    z[i]=z[i]+'/2020'
a=pd.to_datetime(z,dayfirst=True)
df['Date']=a

In [69]:
Lap_list=list(set(df['Laptop ID']))

In [70]:
r2=0
count=0
Name=[]
Price=[]
forecast=[]

In [71]:
def score(result):
    result.plot_predict(start=1,end=11)
    plt.show()
    print(result.predict(start=1,end=11))
    forecast.append(result.predict(start=1,end=11)[-1])

In [72]:
for i in range(len(Lap_list)):
    a=[]
    date=[]
    price=[]
    for j in range(len(df['Laptop ID'])):
        if Lap_list[i]==df['Laptop ID'][j]:
            a.append(j)
            name=df["Name"][j]
            temp_price=df['Price'][j]
            
    Name.append(name)
    Price.append(temp_price)
    for j in a:
        date.append(df['Date'][j])
        price.append(df['Price'][j])    
    r1 = pd.date_range(start='2020-11-13', end='2020-11-18')
    p1=[]
    flag=0
    for k in range(len(r1)):
        for j in range(len(date)):
            if(r1[k]==date[j]):
                p1.append(price[j])
                flag=1
        if(flag==0):
            p1.append(np.nan)
        flag=0
    Product=pd.DataFrame()
    Product['Date']=r1
    Product['Price']=p1
    Product=Product.ffill(axis = 0)
    Product=Product.bfill(axis = 0)
    Product.set_index("Date",inplace=True)
    print(i,Product)
    # Predicting the microsoft stocks volume
    rcParams['figure.figsize'] = 16, 6
    mod_1=0
    mod_2=0
    mod_3=0
    
    model1 = ARIMA(Product['Price'], order=(1,0,2))
    model2 = ARIMA(Product['Price'], order=(1,0,1))
    model3 = ARIMA(Product['Price'], order=(2,0,1))
    try:
        result1 = model1.fit()    
        result1.plot_predict(start=1, end=11)
        plt.show()
        mod_1=r2_score(Product['Price'], result1.predict(start=1,end=6))
        print(mod_1)
    except np.linalg.LinAlgError as l:
        mod_1=0
    except ValueError as v:
        mod_1=0

    try:
        result2 = model2.fit()    
        result2.plot_predict(start=1, end=11)
        plt.show()
        mod_2=r2_score(Product['Price'], result2.predict(start=1,end=6))
        print(mod_2)
    except np.linalg.LinAlgError as l:
        mod_2=0
    except ValueError as v:
        mod_2=0

    try:        
        result3 = model3.fit()    
        result3.plot_predict(start=1, end=11)
        plt.show()
        mod_3=r2_score(Product['Price'], result3.predict(start=1,end=6))
        print(mod_3)
    except np.linalg.LinAlgError as l:
        mod_3=0
    except ValueError as v:
        mod_3=0
    
    if (mod_1>mod_2 and mod_1>mod_3 and mod_1>0.3):
        result=model1.fit()
        r2+=mod_1
        count+=1
        print(mod_1)
        score(result)
    elif(mod_2>mod_3 and mod_2>mod_1 and mod_2>0.3):
        result=model2.fit()
        r2+=mod_2
        count+=1
        print(mod_2)
        score(result)
    elif(mod_3>mod_1 and mod_3>mod_2 and mod_3>0.3):
        result=model3.fit()
        r2+=mod_3
        count+=1
        print(mod_3)
        score(result)
    else:
        forecast.append(temp_price)    

0                Price
Date                
2020-11-13  102605.0
2020-11-14  102605.0
2020-11-15  102605.0
2020-11-16  102605.0
2020-11-17  102605.0
2020-11-18  102605.0
1                Price
Date                
2020-11-13  122900.0
2020-11-14  122900.0
2020-11-15  122900.0
2020-11-16  122900.0
2020-11-17  122900.0
2020-11-18  119900.0
2             Price
Date             
2020-11-13  76000
2020-11-14  75999
2020-11-15  75999
2020-11-16  75999
2020-11-17  75999
2020-11-18  75999
3               Price
Date               
2020-11-13  79990.0
2020-11-14  79990.0
2020-11-15  79990.0
2020-11-16  79990.0
2020-11-17  79990.0
2020-11-18  79990.0
4               Price
Date               
2020-11-13  33790.0
2020-11-14  33790.0
2020-11-15  33790.0
2020-11-16  33790.0
2020-11-17  33790.0
2020-11-18  33790.0
5               Price
Date               
2020-11-13  35290.0
2020-11-14  35290.0
2020-11-15  35290.0
2020-11-16  35290.0
2020-11-17  35290.0
2020-11-18  35290.0
6               Price
Date  

52               Price
Date               
2020-11-13  38490.0
2020-11-14  38490.0
2020-11-15  38490.0
2020-11-16  38490.0
2020-11-17  38490.0
2020-11-18  38490.0
53               Price
Date               
2020-11-13  40990.0
2020-11-14  40990.0
2020-11-15  40990.0
2020-11-16  40990.0
2020-11-17  40990.0
2020-11-18  40990.0
54                Price
Date                
2020-11-13  206440.0
2020-11-14  206440.0
2020-11-15  206440.0
2020-11-16  206440.0
2020-11-17  206440.0
2020-11-18  206440.0
55             Price
Date             
2020-11-13  49490
2020-11-14  49490
2020-11-15  49490
2020-11-16  49490
2020-11-17  49490
2020-11-18  49490
56             Price
Date             
2020-11-13  61790
2020-11-14  61790
2020-11-15  61790
2020-11-16  61790
2020-11-17  61790
2020-11-18  61790
57             Price
Date             
2020-11-13  91290
2020-11-14  91290
2020-11-15  91290
2020-11-16  91290
2020-11-17  91290
2020-11-18  91290
58             Price
Date             
2020-11-13  69321
2020-

106               Price
Date               
2020-11-13  46990.0
2020-11-14  46990.0
2020-11-15  46990.0
2020-11-16  46990.0
2020-11-17  46990.0
2020-11-18  46990.0
107             Price
Date             
2020-11-13  43799
2020-11-14  43799
2020-11-15  43799
2020-11-16  43799
2020-11-17  43990
2020-11-18  43990
108             Price
Date             
2020-11-13  51990
2020-11-14  55592
2020-11-15  55592
2020-11-16  55592
2020-11-17  55592
2020-11-18  51990
109               Price
Date               
2020-11-13  31680.0
2020-11-14  31680.0
2020-11-15  31680.0
2020-11-16  31680.0
2020-11-17  31680.0
2020-11-18  31680.0
110               Price
Date               
2020-11-13  34490.0
2020-11-14  34490.0
2020-11-15  34490.0
2020-11-16  34490.0
2020-11-17  34490.0
2020-11-18  34490.0
111               Price
Date               
2020-11-13  91999.0
2020-11-14  91999.0
2020-11-15  91999.0
2020-11-16  91999.0
2020-11-17  91999.0
2020-11-18  91999.0
112             Price
Date             
2020-11-

158               Price
Date               
2020-11-13  72990.0
2020-11-14  72990.0
2020-11-15  72990.0
2020-11-16  72990.0
2020-11-17  72990.0
2020-11-18  72990.0
159               Price
Date               
2020-11-13  37990.0
2020-11-14  37990.0
2020-11-15  37490.0
2020-11-16  37490.0
2020-11-17  37490.0
2020-11-18  37490.0
160               Price
Date               
2020-11-13  37592.0
2020-11-14  37592.0
2020-11-15  37592.0
2020-11-16  37592.0
2020-11-17  37592.0
2020-11-18  37592.0
161             Price
Date             
2020-11-13  56921
2020-11-14  56921
2020-11-15  56921
2020-11-16  55628
2020-11-17  55598
2020-11-18  55598
162               Price
Date               
2020-11-13  43490.0
2020-11-14  43490.0
2020-11-15  43490.0
2020-11-16  43490.0
2020-11-17  43490.0
2020-11-18  43490.0
163              Price
Date              
2020-11-13  148000
2020-11-14  148000
2020-11-15  148000
2020-11-16  147490
2020-11-17  147490
2020-11-18  147490
164                Price
Date           

212                Price
Date                
2020-11-13  289269.0
2020-11-14  289269.0
2020-11-15  289269.0
2020-11-16  289269.0
2020-11-17  289269.0
2020-11-18  289269.0
213              Price
Date              
2020-11-13  213000
2020-11-14  213000
2020-11-15  213000
2020-11-16  213000
2020-11-17  213000
2020-11-18  213000
214             Price
Date             
2020-11-13  85990
2020-11-14  85990
2020-11-15  85990
2020-11-16  85900
2020-11-17  85900
2020-11-18  85900
215             Price
Date             
2020-11-13  73492
2020-11-14  71992
2020-11-15  71992
2020-11-16  78392
2020-11-17  71992
2020-11-18  71992
216               Price
Date               
2020-11-13  27999.0
2020-11-14  27999.0
2020-11-15  27999.0
2020-11-16  27999.0
2020-11-17  27999.0
2020-11-18  27999.0
217             Price
Date             
2020-11-13  33990
2020-11-14  33990
2020-11-15  33990
2020-11-16  33990
2020-11-17  33990
2020-11-18  33990
218                Price
Date                
2020-11-13  102000

...

1664              Price
Date              
2020-11-13  175900
2020-11-14  175900
2020-11-15  175900
2020-11-16  175900
2020-11-17  175900
2020-11-18  175900
1665               Price
Date               
2020-11-13  32990.0
2020-11-14  32990.0
2020-11-15  33790.0
2020-11-16  33590.0
2020-11-17  33590.0
2020-11-18  33590.0
1666             Price
Date             
2020-11-13  41990
2020-11-14  41990
2020-11-15  41990
2020-11-16  41990
2020-11-17  41990
2020-11-18  41990
1667             Price
Date             
2020-11-13  61690
2020-11-14  61690
2020-11-15  61690
2020-11-16  61690
2020-11-17  61690
2020-11-18  61690
1668               Price
Date               
2020-11-13  66666.0
2020-11-14  66666.0
2020-11-15  66666.0
2020-11-16  66666.0
2020-11-17  66666.0
2020-11-18  66666.0
1669               Price
Date               
2020-11-13  36990.0
2020-11-14  36990.0
2020-11-15  36990.0
2020-11-16  36990.0
2020-11-17  36990.0
2020-11-18  36990.0
1670               Price
Date               
2020-

1717             Price
Date             
2020-11-13  41689
2020-11-14  46989
2020-11-15  46989
2020-11-16  46989
2020-11-17  46989
2020-11-18  46989
1718              Price
Date              
2020-11-13  114000
2020-11-14  114000
2020-11-15  114000
2020-11-16  114000
2020-11-17  114000
2020-11-18  114000
1719               Price
Date               
2020-11-13  40990.0
2020-11-14  40990.0
2020-11-15  40990.0
2020-11-16  40990.0
2020-11-17  40990.0
2020-11-18  40990.0
1720               Price
Date               
2020-11-13  46790.0
2020-11-14  46790.0
2020-11-15  46790.0
2020-11-16  46790.0
2020-11-17  46790.0
2020-11-18  46790.0
1721              Price
Date              
2020-11-13  139990
2020-11-14  139990
2020-11-15  139990
2020-11-16  139990
2020-11-17  139990
2020-11-18  139990
1722               Price
Date               
2020-11-13  59999.0
2020-11-14  59999.0
2020-11-15  59999.0
2020-11-16  59999.0
2020-11-17  59999.0
2020-11-18  59999.0
1723               Price
Date             

1769               Price
Date               
2020-11-13  75490.0
2020-11-14  75490.0
2020-11-15  75490.0
2020-11-16  75490.0
2020-11-17  75490.0
2020-11-18  75490.0
1770               Price
Date               
2020-11-13  58490.0
2020-11-14  58490.0
2020-11-15  58490.0
2020-11-16  58490.0
2020-11-17  58400.0
2020-11-18  58400.0
1771             Price
Date             
2020-11-13  42890
2020-11-14  42890
2020-11-15  42890
2020-11-16  42890
2020-11-17  42890
2020-11-18  42890
1772             Price
Date             
2020-11-13  29490
2020-11-14  29490
2020-11-15  29490
2020-11-16  29490
2020-11-17  29490
2020-11-18  29490
1773               Price
Date               
2020-11-13  27990.0
2020-11-14  27990.0
2020-11-15  27990.0
2020-11-16  27990.0
2020-11-17  27990.0
2020-11-18  27990.0
1774               Price
Date               
2020-11-13  41590.0
2020-11-14  41590.0
2020-11-15  41590.0
2020-11-16  41590.0
2020-11-17  41590.0
2020-11-18  41590.0
1775               Price
Date             

1821               Price
Date               
2020-11-13  32990.0
2020-11-14  32990.0
2020-11-15  32990.0
2020-11-16  32990.0
2020-11-17  32990.0
2020-11-18  32990.0
1822               Price
Date               
2020-11-13  45790.0
2020-11-14  45790.0
2020-11-15  45790.0
2020-11-16  45790.0
2020-11-17  45790.0
2020-11-18  45790.0
1823             Price
Date             
2020-11-13  34999
2020-11-14  34999
2020-11-15  34999
2020-11-16  36999
2020-11-17  36999
2020-11-18  36999
1824               Price
Date               
2020-11-13  97990.0
2020-11-14  97990.0
2020-11-15  97990.0
2020-11-16  97990.0
2020-11-17  97990.0
2020-11-18  97990.0
1825             Price
Date             
2020-11-13  92000
2020-11-14  92000
2020-11-15  92000
2020-11-16  92000
2020-11-17  92000
2020-11-18  92000
1826               Price
Date               
2020-11-13  35490.0
2020-11-14  35490.0
2020-11-15  35490.0
2020-11-16  35490.0
2020-11-17  35490.0
2020-11-18  35490.0
1827               Price
Date             

1875               Price
Date               
2020-11-13  53890.0
2020-11-14  53890.0
2020-11-15  53890.0
2020-11-16  53890.0
2020-11-17  53890.0
2020-11-18  53890.0
1876               Price
Date               
2020-11-13  57682.0
2020-11-14  57682.0
2020-11-15  57682.0
2020-11-16  57682.0
2020-11-17  57682.0
2020-11-18  57682.0
1877               Price
Date               
2020-11-13  43999.0
2020-11-14  43999.0
2020-11-15  43999.0
2020-11-16  43999.0
2020-11-17  43999.0
2020-11-18  43999.0


In [73]:
print(len(Name),len(Price),len(forecast))

1878 1878 1878


In [74]:
acer="https://rukminim1.flixcart.com/image/312/312/khkvukw0/computer/9/e/h/acer-na-thin-and-light-laptop-original-imafxkchtj8zfcev.jpeg?q=70"
asus="https://rukminim1.flixcart.com/image/416/416/k391w280/computer/x/a/h/asus-na-original-imafmewrxvc7ygmb.jpeg?q=70"
lenovo="https://rukminim1.flixcart.com/image/312/312/kgv5x8w0/computer/a/7/u/lenovo-original-imafxyh4chxv6nxq.jpeg?q=70"
dell="https://rukminim1.flixcart.com/image/312/312/ken59jk0/computer/8/w/k/dell-na-thin-and-light-laptop-original-imafv9twss4ck7pk.jpeg?q=70"
hp="https://rukminim1.flixcart.com/image/312/312/k5bcscw0/computer/f/u/j/hp-na-laptop-original-imaferegbpwq9bhc.jpeg?q=70"
apple="https://rukminim1.flixcart.com/image/312/312/k9bo9e80/computer/5/m/6/apple-na-original-imafr5acuz95sa5g.jpeg?q=70"
msi="https://rukminim1.flixcart.com/image/312/312/ke353m80/computer/p/q/m/msi-original-imafuurxjxzy4zcn.jpeg?q=70"
other="https://rukminim1.flixcart.com/image/312/312/kdnf98w0/computer/y/d/5/msi-original-imafugkuq8aa2pkv.jpeg?q=70"

In [78]:
Link=[]
for i in range(len(Name)):
    if "Acer" in Name[i]:
        Link.append(acer)        
    elif "Asus" in Name[i]:
        Link.append(asus)        
    elif "Lenovo" in Name[i]:
        Link.append(lenovo)
    elif "Dell" in Name[i]:
        Link.append(dell)
    elif "HP" in Name[i]:
        Link.append(hp)
    elif "Apple" in Name[i]:
        Link.append(apple)
    elif "MSI" in Name[i]:
        Link.append(msi)
    else:
        Link.append(other)

In [79]:
print(len(Name),len(Price),len(forecast),len(Link))

1878 1878 1878 1878


In [80]:
Forecast_df=pd.DataFrame()
Forecast_df['Name']=Name
Forecast_df['Price']=Price
Forecast_df['Forecast']=forecast
Forecast_df['Image']=Link

In [81]:
Forecast_df["Change"]=Forecast_df['Forecast']-Forecast_df['Price']
Forecast_df['Change in %']=(Forecast_df['Change'] *100)/Forecast_df['Price']
Forecast_df['Forecast']=Forecast_df['Forecast'].round(0)
Forecast_df['Change']=Forecast_df['Change'].round(2)
Forecast_df['Change in %']=Forecast_df['Change in %'].round(1)


In [82]:
Forecast_df

,Name,Price,Forecast,Image,Change,Change in %
0,Smart Buy Z240T WKSTN E3-1240 3.3G 16GB 1TB DV...,102605,102605,https://rukminim1.flixcart.com/image/312/312/k...,0,0.0
1,Microsoft Surface Book 2 Intel Core i5 7th Gen...,119900,119900,https://rukminim1.flixcart.com/image/312/312/k...,0,0.0
2,Acer Nitro AN515-52 Core i5 8th Gen 15.6-inch ...,75999,75999,https://rukminim1.flixcart.com/image/312/312/k...,0,0.0
3,Lenovo ThinkPad L490 Intel Core i7 8th Gen 14-...,79990,79990,https://rukminim1.flixcart.com/image/312/312/k...,0,0.0
4,(Renewed) Lenovo ThinkPad E480 Intel Core i3 8...,33790,33790,https://rukminim1.flixcart.com/image/312/312/k...,0,0.0
...,...,...,...,...,...,...
1873,(Renewed) HP Probook Laptop 440g1 Intel Core i...,44290,44290,https://rukminim1.flixcart.com/image/312/312/k...,0,0.0
1874,ASUS VivoBook 14 Intel Core i7-10510U 10th Gen...,82990,82990,https://rukminim1.flixcart.com/image/312/312/k...,0,0.0
1875,(Renewed) HP Hybrid Probook Laptop 430G3 Intel...,53890,53890,https://rukminim1.flixcart.com/image/312/312/k...,0,0.0
1876,Acer Nitro 5 AN515-43 15.6 inch FHD IPS Displa...,57682,57682,https://rukminim1.flixcart.com/image/312/312/k...,0,0.0


In [88]:
top_picks=Forecast_df.sort_values(by=['Change in %'])
top_picks=top_picks.reset_index()
top_picks=top_picks.drop(columns=['index'])

In [92]:
top=top_picks.iloc[1:7]
top

,Name,Price,Forecast,Image,Change,Change in %
1,(Renewed) HP Probook Laptop 440g1 Intel Core i...,38990,38990,https://rukminim1.flixcart.com/image/312/312/k...,0,0.0
2,(Renewed) Microsoft Surface Laptop 3 AMD Ryzen...,95192,95192,https://rukminim1.flixcart.com/image/312/312/k...,0,0.0
3,HP Envy 13.3-inch Touchscreen FHD Laptop (10th...,78490,78490,https://rukminim1.flixcart.com/image/312/312/k...,0,0.0
4,(Renewed) Lenovo Thinkpad L450 Laptop (Core i5...,29490,29490,https://rukminim1.flixcart.com/image/312/312/k...,0,0.0
5,DELL Latitude 3500 15.6-inch Laptop (8th Gen C...,36799,36799,https://rukminim1.flixcart.com/image/312/312/k...,0,0.0
6,Lenovo Ideapad L340 Gaming Intel Core i5 9th G...,59990,59990,https://rukminim1.flixcart.com/image/312/312/k...,0,0.0


In [ ]:
top.to_json("Top_Picks_Amazon.json")
Forecast_df.to_json("Amazon_Forecast.json")